# AutoGen 

[AutoGen](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/installation.html) is a programming framework for agentic.

Install the dependencies library:

In [1]:
%pip install -qU autogen-agentchat "autogen-ext[openai]" "autogen-ext[ollama]"

Note: you may need to restart the kernel to use updated packages.


## Setup LLM and Agent

In [2]:
import os
from dotenv import load_dotenv
from getpass import getpass

# Load environment variables from .env file
load_dotenv()

# Get API base URL and model with default values
OPENAI_API_BASE = os.getenv("OPENAI_API_BASE", "https://api.openai.com/v1")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o")

# Set up OpenAI API configuration
# Try to get API key from environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# If not found, ask user to input it
if not OPENAI_API_KEY:
    OPENAI_API_KEY = getpass("Enter your OpenAI API key:")

OPENAI_MODEL

'qwen2.5:7b'

In [3]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.models.ollama import OllamaChatCompletionClient
from autogen_core.models import UserMessage

# client = OpenAIChatCompletionClient(
#     model=OPENAI_MODEL,
#     api_key=OPENAI_API_KEY,
#     base_url=OPENAI_API_BASE,
# )

client = OllamaChatCompletionClient(
    model=OPENAI_MODEL
)

result = await client.create([UserMessage(content="tell me a short story",source="user")])
result.content

'Sure! Here\'s a short story for you:\n\n---\n\nIn the heart of a bustling city stood an old bookstore that had seen better days. Its shelves were lined with faded books, and the walls whispered tales of forgotten times. The owner, Mr. Elwood, was known for his vast knowledge and the unwavering belief in the magic of stories.\n\nOne rainy afternoon, a young girl named Lily wandered into the bookstore, her eyes wide with curiosity. She walked between the rows of books, feeling the cool leather bindings and the soft paper pages. Mr. Elwood noticed her and approached, asking if she was looking for something specific.\n\nLily shook her head but smiled, explaining that she often dreamed of a world beyond the ones in these books. "Do you think," she asked hesitantly, "that stories can really change things?"\n\nMr. Elwood nodded warmly, his eyes gleaming with memories. He led her to a book he had recently acquired: "The Chronicles of the Forgotten Realm." As they sat on creaky chairs, he bega

## Running an Agent

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
from autogen_agentchat.ui import Console

agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="Use tools to slove tasks."
)

### Get Response

In [8]:
response = await agent.on_messages(
    [TextMessage(content="What is 1.77+3.23*2.72?", source="user")],
    cancellation_token=CancellationToken(),
)
# inner_messages are agent's "thought process" messages
print(response.inner_messages)
# agent_message is the final message from the assistant
print(response.chat_message)

[]
source='assistant' models_usage=RequestUsage(prompt_tokens=38, completion_tokens=139) metadata={} content='To solve the expression \\(1.77 + 3.23 \\times 2.72\\), follow these steps:\n\n1. First, perform the multiplication:\n   \\[\n   3.23 \\times 2.72 = 8.7936\n   \\]\n\n2. Then add the result to 1.77:\n   \\[\n   1.77 + 8.7936 = 10.5636\n   \\]\n\nSo, \\(1.77 + 3.23 \\times 2.72 = 10.5636\\).' type='TextMessage'


### Streaming Response

In [9]:
await Console(
    agent.on_messages_stream(
        [TextMessage(content="tell me a short story", source="user")],
        cancellation_token=CancellationToken(),
    ),
    output_stats=True,
)

---------- assistant ----------
Sure! Here's a short story for you:

---

In the small town of Willowbrook, there lived an old clockmaker named Eli. His shop was tucked away on a quiet street, and it was said that his clocks never stopped ticking—though they did stop at exactly midnight every night.

One stormy evening, a young girl named Lily knocked on Eli's door, seeking shelter from the rain. As she stood in the warmth of his workshop, Eli noticed her curious gaze fixed on an old, dusty clock. "I can fix it," he offered, pointing to the clock with a wink.

Lily was skeptical but agreed to let Eli try. The next morning, as the sun rose, Lily woke up to find that not only did all the clocks in Willowbrook start working again at midnight, but her own heart seemed lighter. She realized that sometimes, what seems broken can be mended by a little kindness and unexpected magic.

---

I hope you enjoyed this short story!
[Prompt tokens: 191, Completion tokens: 199]
---------- Summary -----

Response(chat_message=TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=191, completion_tokens=199), metadata={}, content='Sure! Here\'s a short story for you:\n\n---\n\nIn the small town of Willowbrook, there lived an old clockmaker named Eli. His shop was tucked away on a quiet street, and it was said that his clocks never stopped ticking—though they did stop at exactly midnight every night.\n\nOne stormy evening, a young girl named Lily knocked on Eli\'s door, seeking shelter from the rain. As she stood in the warmth of his workshop, Eli noticed her curious gaze fixed on an old, dusty clock. "I can fix it," he offered, pointing to the clock with a wink.\n\nLily was skeptical but agreed to let Eli try. The next morning, as the sun rose, Lily woke up to find that not only did all the clocks in Willowbrook start working again at midnight, but her own heart seemed lighter. She realized that sometimes, what seems broken can be mended by a little kindness and unexpec

### Streaming Tokens

In [14]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="Use tools to slove tasks.",
    model_client_stream=True, # Enable streaming tokens
)

async for message in agent.on_messages_stream(
    [TextMessage(content="tell me a short story", source="user")],
    cancellation_token=CancellationToken(),
):
    print(message)

source='assistant' models_usage=None metadata={} content='Once' type='ModelClientStreamingChunkEvent'
source='assistant' models_usage=None metadata={} content=' upon' type='ModelClientStreamingChunkEvent'
source='assistant' models_usage=None metadata={} content=' a' type='ModelClientStreamingChunkEvent'
source='assistant' models_usage=None metadata={} content=' time' type='ModelClientStreamingChunkEvent'
source='assistant' models_usage=None metadata={} content=',' type='ModelClientStreamingChunkEvent'
source='assistant' models_usage=None metadata={} content=' in' type='ModelClientStreamingChunkEvent'
source='assistant' models_usage=None metadata={} content=' a' type='ModelClientStreamingChunkEvent'
source='assistant' models_usage=None metadata={} content=' small' type='ModelClientStreamingChunkEvent'
source='assistant' models_usage=None metadata={} content=' village' type='ModelClientStreamingChunkEvent'
source='assistant' models_usage=None metadata={} content=' nestled' type='ModelCli

Similary, `run_stream()` will also yiled the same streming chunks:

In [18]:
from autogen_agentchat.messages import ModelClientStreamingChunkEvent

async for message in agent.run_stream(task="tell me a short story"):
    if isinstance(message, TextMessage):
        # print(message.content)
        pass
    elif isinstance(message, ModelClientStreamingChunkEvent):
        print(message.content, end="")
    else:
        # Handle other message types if needed
        # For example, if you have a custom message type
        # you can process it accordingly
        # print("Other message type:", message)
        pass

Sure! Here’s a short story for you:

---

In a small village nestled among rolling hills and winding rivers, there lived an old watchmaker named Harold. Known for his precise craftsmanship and kind heart, Harold spent most of his days mending clocks and watches, ensuring they kept perfect time.

One rainy afternoon, a young boy named Timmy wandered into Harold's shop. Timmy was curious about the intricate gears and cogs that filled the room, each ticking rhythm creating a soothing melody in the background.

"Hello there," Harold said with a warm smile. "What brings you to my shop on such a dreary day?"

Timmy held up his wristwatch, which had stopped working during the rainstorm. "My watch isn't working anymore. Can you fix it?" he asked hopefully.

Harold took the watch and examined it carefully. After some gentle adjustments, the watch sprang back to life, its hands moving smoothly again.

"Thank you so much!" Timmy exclaimed, beaming with delight.

Harold handed him the watch and sa

## Tools

### Function tools

In [19]:
def add_tool(x: float, y: float) -> float:
    """
    Add X and Y to get the exact result.

    Args:
        x (float): First number.
        y (float): Second number.

    Returns:
        float: The sum of X and Y.
    """
    return x + y

def subtract_tool(x: float, y: float) -> str:
    """
    Subtract Y from X to get the exact result.

    Args:
        x (float): First number.
        y (float): Second number.

    Returns:
        str: The difference of X and Y.
    """
    return str(x - y)

def multiply_tool(x: float, y: float) -> str:
    """
    Multiply X and Y to get the exact result.

    Args:
        x (float): First number.
        y (float): Second number.

    Returns:
        str: The product of X and Y.
    """
    return str(x * y)

def divide_tool(x: float, y: float) -> str:
    """
    Divide X by Y to get the exact result.

    Args:
        x (float): First number.
        y (float): Second number.

    Returns:
        str: The quotient of X and Y.
    """
    return str(x / y)

### Run an Agent with tools

In [20]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[add_tool, subtract_tool, multiply_tool, divide_tool],
    system_message="Use tools to slove tasks."
)

await Console(
    agent.on_messages_stream(
        [TextMessage(content="What is 1.77+3.23*2.72?", source="user")],
        cancellation_token=CancellationToken(),
    ),
    output_stats=True,
)

---------- assistant ----------
[FunctionCall(id='3', arguments='{"x": 3.23, "y": 2.72}', name='multiply_tool')]
[Prompt tokens: 547, Completion tokens: 100]
---------- assistant ----------
[FunctionExecutionResult(content='8.7856', name='multiply_tool', call_id='3', is_error=False)]
---------- assistant ----------
8.7856
---------- Summary ----------
Number of inner messages: 2
Total prompt tokens: 547
Total completion tokens: 100
Duration: 4.43 seconds


Response(chat_message=ToolCallSummaryMessage(source='assistant', models_usage=None, metadata={}, content='8.7856', type='ToolCallSummaryMessage'), inner_messages=[ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=547, completion_tokens=100), metadata={}, content=[FunctionCall(id='3', arguments='{"x": 3.23, "y": 2.72}', name='multiply_tool')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='8.7856', name='multiply_tool', call_id='3', is_error=False)], type='ToolCallExecutionEvent')])

### Conversational Agent

`AssistantAgent` can use `model_context` to pass in a `ChatCompletionContext` object. It allows the agent to use different model contexts. By default, `AssistantAgent` uses the `UnboundedChatCompletionContext` which sends the full conversation history to the model. To limit the context to the last `n` messages, you can use the `BufferedChatCompletionContext`. To limit the context by token count, you can use the `TokenLimitedChatCompletionContext`.

In [32]:
from autogen_core.model_context import BufferedChatCompletionContext

agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[add_tool, subtract_tool, multiply_tool, divide_tool],
    system_message="Use tools to slove tasks.",
    model_context=BufferedChatCompletionContext(buffer_size=2),
)

await Console(
    agent.on_messages_stream(
        [TextMessage(content="a=1.3*2, tell me a=?", source="user")],
        cancellation_token=CancellationToken(),
    ),
    output_stats=True,
)
await Console(
    agent.on_messages_stream(
        [TextMessage(content="b=1.3*a, What is b?", source="user")],
        cancellation_token=CancellationToken(),
    ),
    output_stats=True,
)

---------- assistant ----------
[FunctionCall(id='20', arguments='{"x": 1.3, "y": 2}', name='multiply_tool')]
[Prompt tokens: 541, Completion tokens: 28]
---------- assistant ----------
[FunctionExecutionResult(content='2.6', name='multiply_tool', call_id='20', is_error=False)]
---------- assistant ----------
2.6
---------- Summary ----------
Number of inner messages: 2
Total prompt tokens: 541
Total completion tokens: 28
Duration: 0.66 seconds
---------- assistant ----------
To help you calculate the value of `b`, I need to know the value of `a`. Could you please provide me with the value of `a`?
[Prompt tokens: 540, Completion tokens: 34]
---------- Summary ----------
Number of inner messages: 0
Total prompt tokens: 540
Total completion tokens: 34
Duration: 0.64 seconds


Response(chat_message=TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=540, completion_tokens=34), metadata={}, content='To help you calculate the value of `b`, I need to know the value of `a`. Could you please provide me with the value of `a`?', type='TextMessage'), inner_messages=[])

Since `a=1.3*2` is required to call Function Call, in essence it has a dialog that generates Function Call and then calls Function Call, so when the buffer size is 2, it only has 2.6 and the notation of executing function exectution, and it doesn't know what a is anymore.

In [34]:
from autogen_core.model_context import BufferedChatCompletionContext

agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[add_tool, subtract_tool, multiply_tool, divide_tool],
    system_message="Use tools to slove tasks.",
    model_context=BufferedChatCompletionContext(buffer_size=3),
)

await Console(
    agent.on_messages_stream(
        [TextMessage(content="a=1.3*2, what is a?", source="user")],
        cancellation_token=CancellationToken(),
    ),
    output_stats=True,
)
await Console(
    agent.on_messages_stream(
        [TextMessage(content="b=1.3*a, What is b?", source="user")],
        cancellation_token=CancellationToken(),
    ),
    output_stats=True,
)

---------- assistant ----------
[FunctionCall(id='23', arguments='{"x": 1.3, "y": 2}', name='multiply_tool')]
[Prompt tokens: 541, Completion tokens: 28]
---------- assistant ----------
[FunctionExecutionResult(content='2.6', name='multiply_tool', call_id='23', is_error=False)]
---------- assistant ----------
2.6
---------- Summary ----------
Number of inner messages: 2
Total prompt tokens: 541
Total completion tokens: 28
Duration: 2.03 seconds
---------- assistant ----------
<tool_call>
{"name": "multiply_tool", "arguments": {"x": 1.3, "y": a}}
</tool_call>
[Prompt tokens: 586, Completion tokens: 27]
---------- Summary ----------
Number of inner messages: 0
Total prompt tokens: 586
Total completion tokens: 27
Duration: 0.58 seconds


Response(chat_message=TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=586, completion_tokens=27), metadata={}, content='<tool_call>\n{"name": "multiply_tool", "arguments": {"x": 1.3, "y": a}}\n</tool_call>', type='TextMessage'), inner_messages=[])